# Scraping: http://www.bbc.co.uk/news

Let's try to scrape the frontpage of BBC News. We're looking for

* Headlines
* Summary
* Article link

## Getting started

We'll start by **importing the necessary libraries**.

And then move into **downloading the page** and **importing it into BeautifulSoup**.

A lot of people call the analyzed page variable `soup` but for once in my life I actually go against the popular thing - I like to call it `doc`, since it helps me remember that it's the *entire document*.

## ATTEMPT ONE: Grabbing the tags directly

If we look at the page, we try to use the little arrow-selecty-thing to pick up the headlines and **disaster strikes**. We can't touch it! Apparently it's the ENTIRE BLOCK or something crazy like that?

But luckily we understand HTML, so we can click around on the right-hand Elements page. We navigate to the `h3` tag, which we know is the headline based on the tag name and the content.

Hm, what if we just grab all of the `h3` tags?

SO EASY, right? Kind of? Mostly it worked? ...except it doesn't have the link, nor does it have the summary.

Okay, so we could also get all of the `a` tags, but there are probably a lot of garbage `a` tags - footer content and stuff. Maybe the article `a` tags have a special class? If we take a look, we see `class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold"`. This isn't just one class, it's **many classes**.

* `gs-c-promo-heading`
* `nw-o-link-split__anchor`
* `gs-o-faux-block-link__overlay-link`
* `gel-pica-bold`

This is where guesswork comes ib. I think `gs-c-promo-heading` seems reasonable!

That looks pretty good, too! It's getting the `h3` text because the `h3` is inside of the `a` tag, but it doesn't have the *actual link*, the URL. If we look at the `a` tag...

    <a class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold" href="/news/world-middle-east-39302560">
   
...the URL is hiding in the `href` attribute. Once we have the link, it's actually easy to get an attribute, you just use `['href']`

Cool, 'eh? But now we have one final problem: **we don't have the summaries**. So well, we can just use the Inspector to pick one out...

    <p class="gs-c-promo-summary gel-long-primer gs-u-mt nw-c-promo-summary">Trade and Nato are high on the agenda as the much-anticipated Washington talks begin.</p>

Once again, we have a selection of options. `gs-c-promo-summary` seems promising.

Great, but **now we're stuck:** we don't have a way of combining the headlines and links to the summaries, and even if we did (cough`zip`cough), we couldn't be sure that they'd match up.

What the heck do we do now?

## ATTEMPT TWO: Parent elements

When you're just grabbing one element - a link and the text inside, or a list of headlines - you are only interested in the element you're looking at. Sometimes, though, **you need to scrape multiple elements at the same time.** When this happens, you need to look at what they all have in common.

If we look at a summary, a link and a title, we might find something like the following. **It's a trainwreck, but it's what we want.**

	<div class="gs-c-promo nw-c-promo gs-o-faux-block-link gs-u-pb gs-u-pb+@m gs-c-promo--inline gs-c-promo--stacked@m nw-u-w-auto gs-c-promo--flex" data-entityid="container-top-stories#3">
		<div class="gs-c-promo-image gs-u-display-none gs-u-display-inline-block@xs gel-1/2@xs gel-1/1@m">
			<div class="gs-o-media-island">
				<div class="gs-o-responsive-image gs-o-responsive-image--16by9"><img alt="Yemeni police gather round bodies of Somali refugees (17/03/17)" class="qa-lazyload-image lazyautosizes lazyloaded" data-sizes="auto" data-srcset="https://ichef.bbci.co.uk/live-experience/cps/240/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 240w, https://ichef.bbci.co.uk/live-experience/cps/320/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 320w, https://ichef.bbci.co.uk/live-experience/cps/480/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 480w, https://ichef.bbci.co.uk/live-experience/cps/624/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 624w, https://ichef.bbci.co.uk/live-experience/cps/800/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 800w" data-widths="[240,320,480,624,800]" sizes="177px" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" srcset="https://ichef.bbci.co.uk/live-experience/cps/240/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 240w, https://ichef.bbci.co.uk/live-experience/cps/320/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 320w, https://ichef.bbci.co.uk/live-experience/cps/480/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 480w, https://ichef.bbci.co.uk/live-experience/cps/624/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 624w, https://ichef.bbci.co.uk/live-experience/cps/800/cpsprodpb/14F5A/production/_95205858_hi038526789.jpg 800w"></div>
			</div>
		</div>
		<div class="gs-c-promo-body gel-1/2@xs gel-1/1@m gs-u-mt@m">
			<div>
				<a class="gs-c-promo-heading nw-o-link-split__anchor gs-o-faux-block-link__overlay-link gel-pica-bold" href="/news/world-middle-east-39302560">
				<h3 class="gs-c-promo-heading__title gel-pica-bold nw-o-link-split__text">Attack on Yemen migrant boat kills 42</h3></a>
				<p class="gs-c-promo-summary gel-long-primer gs-u-mt nw-c-promo-summary">It is unclear who was behind a helicopter attack which killed 42 refugees and injured 80.</p>
			</div>
			<ul class="gs-o-list-inline gs-o-list-inline--divided gel-brevier gs-u-mt-">
				<li><span class="gs-c-timestamp gs-o-bullet gs-o-bullet- nw-c-timestamp"><span class="gs-o-bullet__icon gel-icon"><svg viewbox="0 0 32 32">
				<polygon points="17,15.4 17,6 15,6 15,16.6 23.8,21.7 24.8,19.9"></polygon>
				<path d="M16,4c6.6,0,12,5.4,12,12c0,6.6-5.4,12-12,12S4,22.6,4,16C4,9.4,9.4,4,16,4 M16,0C7.2,0,0,7.2,0,16c0,8.8,7.2,16,16,16 s16-7.2,16-16C32,7.2,24.8,0,16,0L16,0z"></path></svg></span><time class="gs-o-bullet__text date qa-status-date relative-time" data-datetime="1h" data-seconds="1489768430" data-timestamp-inserted="true" datetime="2017-03-17T16:33:50.000Z">48 minutes ago</time></span></li>
				<li>
					<a aria-label="From Middle East" class="gs-c-section-link gs-c-section-link--truncate nw-c-section-link nw-o-link nw-o-link--no-visited-state" href="/news/world/middle_east"><span aria-hidden="true">Middle East</span></a>
				</li>
			</ul>
		</div>
	</div>

The very top part is the **parent element**, all of the other elements are inside of it. In order to scrape them all together, we need to grab each parent (each *story*) and then grab the parts inside of it (the headline, links, image, etc).

The part's class is `class="gs-c-promo nw-c-promo gs-o-faux-block-link gs-u-pb gs-u-pb+@m gs-c-promo--inline gs-c-promo--stacked@m nw-u-w-auto gs-c-promo--flex" data-entityid="container-top-stories#3"`, which would be terrifying except that we've struck onto a theme and suspect `gs-c-promo` might be what we're looking for.

In [ ]:
stories = doc.find_all('div', { 'class': 'gs-c-promo' })
for story in stories:
    print(story.text)

So... kind of?

We apparently can't use `.text` because it's going to get take *all* of the text inside, it's going to take the headline *and* the summary. What we need to do instead is

* STEP ONE: Use the doc to get the story
* STEP TWO: Use the story to get the headline
* STEP THREE: Use the story to get the link
* STEP FOUR: Use the story to get the summary

### STEP ONE: Use the doc to get the story

## STEP TWO: Use the story to get the headline

Now we can do the same thing to find the link, and then use `['href']` to grab the link URL.

## STEP FOUR: Use the story to get the summary

Same thing again! This time we're looking for a `p`.

### Missing elements

Oh god, an error! If you weren't paying attention, the error is

    ---------------------------------------------------------------------------
    AttributeError                            Traceback (most recent call last)
    <ipython-input-20-e55795264040> in <module>()
          7     print(link['href'])
          8     summary = story.find('p')
    ----> 9     print(summary.text)

    AttributeError: 'NoneType' object has no attribute 'text'

Since it showed up after we added in the `summary` part, I'm going to assume this is an issue because **not every story has a summary**. How do we get around it!!!

Well, just *ask if it has a summary*. If it does, you can use it. If it doesn't, ignore it. **It's just a simple `if` statement**.

## Turning it into a CSV

Now that we have all of our elements, we can turn it into a CSV. There are three steps to building the CSV:
    
1. **Start with an empty list:** Each story we'll find, we'll add it to the list
2. **Build a dictionary** for each story element
3. **Convert the list to a DataFrame**, and then
4. **Export the DataFrame to a CSV**

The dictionary-buiding part can be complicated, so let's look at **two different ways of doing it**.

### Method One: All at once

For this method, we'll make our `story_dict` all at once, then add it to the `stories_list`.

### Method Two: Filling in the blanks

For this method, we'll make our `story_dict` in the beginning, then fill in any pieces that exist.